In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
%%capture
!pip install -q transformers

from __future__ import division
from __future__ import print_function

from google.colab import drive
from google.colab import files

import collections
import copy
import os
import shutil
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torchvision.io import read_image
from torch.utils.data import Dataset

from tqdm.notebook import tqdm

In [ ]:
data_path = '/content/data/'
!rm -rf $data_path
!mkdir -p $data_path

drive_path = '/content/drive/'
drive.mount(drive_path)
project_path = os.path.join(drive_path, 'MyDrive/dissertation')  #project folder
kaggle_path = os.path.join(project_path, '.kaggle')

kaggle_credentials_path = '/root/.kaggle'
!mkdir -p $kaggle_credentials_path
%cd $kaggle_credentials_path

if not os.path.exists(os.path.join(kaggle_credentials_path, 'kaggle.json')):
    if not os.path.exists(os.path.join(kaggle_path, 'kaggle.json')):
        files.upload() #this will prompt you to update the json
    shutil.copy(os.path.join(kaggle_path, 'kaggle.json'), kaggle_credentials_path)
    
!pip install -q kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!ls $kaggle_credentials_path
!chmod 600 $kaggle_credentials_path/kaggle.json  # set permission
!kaggle datasets list

In [ ]:
%cd $data_path
dataset_path = 'covid19pneumonianormal-covidx-cxr2.zip'
!kaggle datasets download -d inesfilipe/covid19pneumonianormal-covidx-cxr2
!unzip -n $dataset_path

In [ ]:
class COVIDxDataset(Dataset):
    def __init__(self, root, metadata, transform=None, target_transform=None, train=True):
        self.root = root
        self.metadata = metadata
        self.transform = transform
        self.target_transform = target_transform
        self.class_to_idx = {label: i for i, label in enumerate(np.unique(metadata[:,2]))}
        self.classes = sorted([label for label in self.class_to_idx])
        self.targets = [self.class_to_idx[i] for i in metadata[:,2]]
        self.imgs = [(os.path.join(self.root, img[2], img[1]), self.class_to_idx[img[2]]) for img in metadata]
        self.train = train
        
    def __len__(self):
        return len(self.metadata)
    
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert("RGB")
    
    def accimage_loader(self, path):
        import accimage

        try:
            return accimage.Image(path)
        except OSError:
            return self.pil_loader(path)

    def default_loader(self, path):
        from torchvision import get_image_backend

        if get_image_backend() == "accimage":
            return self.accimage_loader(path)
        else:
            return self.pil_loader(path)

    def __getitem__(self, idx):
        sample = self.imgs[idx]
        img_path = sample[0]
        target = sample[1]

        image = self.default_loader(img_path)

        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            target = self.transform(target)

        return image, target

In [ ]:
data_transforms = {
        "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ]),
        "test": transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])
}

train_metadata = np.genfromtxt("train.txt", delimiter=" ", dtype=None, encoding="utf-8")
test_metadata = np.genfromtxt("test.txt", delimiter=" ", dtype=None, encoding="utf-8")

training_dataset = COVIDxDataset("train", train_metadata, data_transforms["train"], train=True)
test_dataset = COVIDxDataset("test", test_metadata, data_transforms["test"], train=False)

def make_sampler(dataset, balance=True):
    def make_weights_for_balanced_classes(dataset):
        target_count = collections.Counter(dataset.targets)
        total = sum(target_count.values())
        target_weight = {k : sum(target_count.values()) / v for (k, v) in target_count.items()}
        weight = [0.] * total
    
        for index, value in enumerate(dataset.imgs):
            weight[index] = target_weight[value[1]]
        return weight
  
    if balance:
        weights = torch.DoubleTensor(make_weights_for_balanced_classes(dataset))
        return torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
    return None

def make_loader(dataset, batch_size, sampler=None):
    if sampler == None:
        return torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           pin_memory=True, 
                                           num_workers=2)
    return torch.utils.data.DataLoader(dataset=dataset,
                                       batch_size=batch_size, 
                                       sampler=sampler,
                                       pin_memory=True, 
                                       num_workers=2)

train_loader = make_loader(training_dataset, batch_size=64)
test_loader = make_loader(test_dataset, batch_size=64)

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(test_dataset.classes))

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
        model.parameters(), lr=5e-5)

In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)

In [ ]:
train_accu = {}
train_losses = {}

In [ ]:
def train(epoch):
  print(f"Epoch: {epoch}")
  
  model.train()

  running_loss = 0
  correct = 0
  total = 0

  for idx, (images, labels) in enumerate(tqdm(train_loader)):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    total += labels.shape[0]
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()

    if idx % 100 == 0:
      print(f"Total = {total}")
      print(f"Loss after {idx} steps: {loss.item()}")
      print(f"Accuracy after {idx} steps: {correct / total}")
  
  train_loss = running_loss / len(train_loader)
  accu = 100. * correct / total
  
  train_accu[epoch] = accu
  train_losses[epoch] = train_loss
  print('Train Loss: %.3f | Accuracy: %.3f' % (train_loss, accu))

In [ ]:
eval_accu = {}
eval_losses = {}
confusion_matrices = {}
binary_confusion_matrices = {}
per_class_metrics = {}
micro_metrics = {}
macro_metrics = {}
weighted_metrics = {}
total_preds = {}
total_labels = {}

In [ ]:
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, precision_recall_fscore_support

def test(epoch):
  model.eval()
  
  with torch.no_grad():
    running_loss = 0
    correct = 0
    total = 0

    for _, (images, labels) in enumerate(tqdm(test_loader)):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      
      loss = criterion(outputs, labels)
      running_loss += loss.item()
      total += labels.shape[0]
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels).sum().item()

      if epoch in total_preds:
        total_preds[epoch] = torch.cat((total_preds[epoch], predicted))
      else:
        total_preds[epoch] = predicted
      
      if epoch in total_labels:
        total_labels[epoch] = torch.cat((total_labels[epoch], labels))
      else:
        total_labels[epoch] = labels
      
    test_loss = running_loss / len(test_loader)
    accu = 100. * correct / total
    
    eval_accu[epoch] = accu
    eval_losses[epoch] = test_loss
    total_labels[epoch] = total_labels[epoch].cpu()
    total_preds[epoch] = total_preds[epoch].cpu()
    confusion_matrices[epoch] = confusion_matrix(total_labels[epoch], total_preds[epoch], labels=[0,1,2])
    binary_confusion_matrices[epoch] = multilabel_confusion_matrix(total_labels[epoch], total_preds[epoch], labels=[0,1,2])

    per_class_metrics[epoch] = precision_recall_fscore_support(total_labels[epoch], total_preds[epoch], labels=[0,1,2])
    micro_metrics[epoch] = precision_recall_fscore_support(total_labels[epoch], total_preds[epoch], labels=[0,1,2], average='micro')
    macro_metrics[epoch] = precision_recall_fscore_support(total_labels[epoch], total_preds[epoch], labels=[0,1,2], average='macro')
    weighted_metrics[epoch] = precision_recall_fscore_support(total_labels[epoch], total_preds[epoch], labels=[0,1,2], average='weighted')

    print(f"Total = {total}")
    print(f"Accuracy {correct / total}")

In [ ]:
epochs = 20

for epoch in range(1, epochs+1):
  train(epoch)
test(epochs+1)

In [ ]:
print(train_accu)
print(train_losses)
print(eval_accu)
print(eval_losses)
print(confusion_matrices)
print(binary_confusion_matrices)
print(per_class_metrics)
print(micro_metrics)
print(macro_metrics)
print(weighted_metrics)
print(test_dataset.class_to_idx)
print(total_preds)
print(total_labels)